In [25]:
import pymysql
import json
import re

In [54]:
connect = pymysql.connect("13.230.161.175","public-root","thisisasamplepassword","game_chatbot")
# connect = pymysql.connect("140.134.241.171","hidochunk","**********","game_chatbot")

In [55]:
with open('Steam1201.json') as json_data:
    data = json.load(json_data)

In [56]:
cursor = connect.cursor()

In [57]:
gameTypeAlreadyHave = {}
gameDevelopeCompanyAlreadyHave = {}

for col in data:
    gameName = col["遊戲名稱"]
    gamePrice = col["價錢"]
    gameIntroduction = col["遊戲介紹"]
    gameNearlyScore = col["最近評論"]
    gameTotalScore = col["所有評論"]
    gameReleaseDate = col["發行日期"]
    gameDevelopeCompany = col["公司"]
    gameType = col["標籤"]
    gameLinkURL = col["連結"]

#     評價
    if gameNearlyScore not in ["壓倒性負評","極度負評","大多負評","負評","褒貶不一","好評","大多好評","極度好評","壓倒性好評"]:
        if gameTotalScore not in ["壓倒性負評","極度負評","大多負評","負評","褒貶不一","好評","大多好評","極度好評","壓倒性好評"]:
            gameScore = "NULL"
        else:
            gameScore = gameTotalScore
    else:
        gameScore = gameNearlyScore
        
#     價格
    if "免費" not in gamePrice:
        priceCut = gamePrice.split()
        if priceCut[0] == "NT$":
            gamePriceCurrency = "TWD"
        gamePriceNum = int(priceCut[1].replace(',',''))
    else:
        gamePriceNum = 0
        gamePriceCurrency = "TWD"

    
#     發布日期
    gameReleaseDateCut = gameReleaseDate.split()
    gameReleaseDateYear = gameReleaseDateCut[0]
    gameReleaseDateMonth = gameReleaseDateCut[2]
    gameReleaseDateDay = gameReleaseDateCut[4]
    gameReleaseDateParsed = gameReleaseDateYear+"-"+gameReleaseDateMonth+"-"+gameReleaseDateDay
    
    
    inputValueString = "'"+re.escape(gameName).strip()+"','"+re.escape(gameIntroduction).strip()+"',"+str(gamePriceNum)+",'"+gamePriceCurrency+"','"+gameLinkURL+"','"+gameReleaseDateParsed+"'"
    if gameScore == "NULL":
        inputValueString = inputValueString + ",NULL"
    else:
        inputValueString = inputValueString + ",'" + gameScore +"'"
        
#     game table INSETR
    print("INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ("+inputValueString+");")
    print()
    cursor.execute("INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES (" + inputValueString + ");")
    connect.commit()   
    
    cursor.execute("SELECT @@IDENTITY;")
    gameIDs = cursor.fetchall()
    gameID = gameIDs[0][0]
    
#     標籤
    gameTypeList = gameType.split(",")
    gameTypeList = list(set(gameTypeList))
    for gametype in gameTypeList:
        gametype = gametype.strip()
        if gametype not in gameTypeAlreadyHave:
            print("INSERT INTO type(type_name) VALUES ('"+gametype+"');")
            cursor.execute("INSERT INTO type(type_name) VALUES ('"+gametype+"');")
            connect.commit()
            cursor.execute("SELECT @@IDENTITY;")
            gametypeID = cursor.fetchall()
            gameTypeAlreadyHave[gametype] = gametypeID[0][0]
        print(gameID,",",gameTypeAlreadyHave[gametype])
        cursor.execute("INSERT INTO game_has_type(game_ID,type_ID) VALUES ("+str(gameID)+","+str(gameTypeAlreadyHave[gametype])+");")

#     廠商
    gameDevelopeCompanyList = gameDevelopeCompany.split(",")
    gameDevelopeCompanyList = list(set(gameDevelopeCompanyList))
    for gamedevelopecompany in gameDevelopeCompanyList:
        gamedevelopecompany = gamedevelopecompany.strip()
        if gamedevelopecompany not in gameDevelopeCompanyAlreadyHave:    
            print("INSERT INTO company(company_name) VALUES ('"+gamedevelopecompany+"');")
            cursor.execute("INSERT INTO company(company_name) VALUES ('"+gamedevelopecompany+"');")
            connect.commit()
            cursor.execute("SELECT @@IDENTITY;")
            results = cursor.fetchall()
            gameDevelopeCompanyAlreadyHave[gamedevelopecompany] = results[0][0]
        cursor.execute("INSERT INTO company_has_game(game_ID,company_ID) VALUES ("+str(gameID)+","+str(gameDevelopeCompanyAlreadyHave[gamedevelopecompany])+");")
    
    print(gameName+", "+gamePriceCurrency+str(gamePriceNum)+", "+gameScore)
    print(gameLinkURL)
    print(gameDevelopeCompanyList)
    print(gameTypeList)
    print(gameReleaseDateParsed)
    print()
                       
print(gameTypeAlreadyHave)
print(gameDevelopeCompanyAlreadyHave)
connect.close()

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('STEINS\;GATE','\由5pb\.\和nitroplus\共\同\開\發\的\《STEINS\;GATE\》\曾\被Famitsu\雜\誌\授\予\年\度\優\秀\遊\戲\獎\項\，\並\在Famitsu\讀\者\調\查\中\被\評\為\最\煽\情\的\“\催\淚\遊\戲\”\。\如\此\扣\人\心\弦\的\視\覺\小\說\遊\戲\，\現\已\被\改\編\為\漫\畫\、\動\畫\及\動\畫\電\影\等\。',670,'TWD','https://store.steampowered.com/app/412830/STEINSGATE/?snr=1_7_7_230_150_4','2016-9-8','壓倒性好評');

INSERT INTO type(type_name) VALUES ('單人');
1 , 1
INSERT INTO type(type_name) VALUES ('驚悚');
1 , 2
INSERT INTO type(type_name) VALUES ('休閒');
1 , 3
INSERT INTO type(type_name) VALUES ('劇情豐富');
1 , 4
INSERT INTO type(type_name) VALUES ('戀愛模擬');
1 , 5
INSERT INTO type(type_name) VALUES ('冒險');
1 , 6
INSERT INTO type(type_name) VALUES ('時光旅行');
1 , 7
INSERT INTO type(type_name) VALUES ('懸疑');
1 , 8
INSERT INTO type(type_name) VALUES ('恐怖');
1 , 9
INSERT INTO type(type_name) VALUES ('愛情');
1 , 10
INSERT INTO type(type_name) VALUES ('懸疑驚悚');
1 , 11
IN

6 , 64
6 , 36
6 , 16
6 , 25
6 , 26
6 , 3
INSERT INTO type(type_name) VALUES ('恐龍');
6 , 65
6 , 6
6 , 29
6 , 42
6 , 28
INSERT INTO type(type_name) VALUES ('資源管理');
6 , 66
6 , 14
6 , 46
6 , 1
6 , 47
INSERT INTO type(type_name) VALUES ('城市營造');
6 , 67
INSERT INTO company(company_name) VALUES ('Frontier Developments');
Jurassic World Evolution, TWD1600, 極度好評
https://store.steampowered.com/app/648350/Jurassic_World_Evolution/?snr=1_7_7_230_150_4
['Frontier Developments']
['科幻', '建造', '模擬', '暴力', '多人', '好評原聲帶', '策略', '管理', '休閒', '恐龍', '冒險', '沙盒', '開放世界', '經濟', '資源管理', '氛圍', '生存', '單人', '動作', '城市營造']
2018-6-12

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('Mirror','\這\是\一\款\“\美\少\女\＋\寶\石\消\除\＋GALGAME\”\結\合\的\遊\戲\。\你\在\异\世\界\里\邂\逅\各\种\各\样\的\美\女\，\与\她\们\战\斗\时\你\套\路\深\似\海\。\这\些\自\認\高\貴\的\盜\賊\精\靈\，\優\柔\寡\斷\的\龍\女\僕\，\喜\歡\著\同\性\的\女\王\，\圈\養\面\首\的\森\林\女\獵\手\都\被\你\不\擇\手\段\地\一\一\捕\獲\并\进\行\令\人\窒\息\的\操\作\～',45,'TWD','http

INSERT INTO company(company_name) VALUES ('Feral Interactive (Linux)');
INSERT INTO company(company_name) VALUES ('Feral interactive (Mac)');
Life is Strange - Episode 1, TWD0, 壓倒性好評
https://store.steampowered.com/app/319630/Life_is_Strange__Episode_1/?snr=1_7_7_230_150_4
[' Feral Interactive (Linux)', 'Square Enix', ' Feral interactive (Mac)']
['單人', '獨立', '步行模擬', '休閒', '女主人翁', '劇情豐富', '冒險', '時光旅行', '懸疑', '點擊', '時空操控', '短片', '氛圍', '多結局', '好評原聲帶', '網路爆紅', '動作', '選擇導向', '影集', '第三人稱視角']
2015-1-30

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('\江\湖\求\生\ Ganghood\ Survival','\《Ganghood\ Survival\》\ is\ a\ hardcore\ competitive\ action\ game\ based\ on\ Chinese\ Wushu\ theme\.\ Dozens\ of\ players\ are\ in\ the\ same\ map\,\ and\ play\ as\ a\ \‘Knight\-errant\’\ with\ nothing\ at\ the\ beginning\.\ They\ need\ to\ strengthen\ themselves\ by\ collecting\ resources\ and\ learning\ wushu\ books\,\ beating\ othe

20 , 68
20 , 22
INSERT INTO type(type_name) VALUES ('生存恐怖');
20 , 124
20 , 91
INSERT INTO type(type_name) VALUES ('玩家對戰');
20 , 125
20 , 6
20 , 29
20 , 42
20 , 74
20 , 43
20 , 45
20 , 44
20 , 35
20 , 46
20 , 89
20 , 37
20 , 47
20 , 77
INSERT INTO company(company_name) VALUES ('Facepunch Studios');
Rust, TWD498, 極度好評
https://store.steampowered.com/app/252490/Rust/?snr=1_7_7_230_150_4
['Facepunch Studios']
['多人', '建造', '搶先體驗', '模擬', '生存恐怖', '合作', '玩家對戰', '冒險', '沙盒', '開放世界', '裸露', '第一人稱射擊', '射擊', '末日', '獨立', '生存', '線上合作', '第一人稱視角', '動作', '工藝']
2018-2-9

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('Slime\ Rancher','Slime\ Rancher\ is\ the\ tale\ of\ Beatrix\ LeBeau\,\ a\ plucky\,\ young\ rancher\ who\ sets\ out\ for\ a\ life\ a\ thousand\ light\ years\ away\ from\ Earth\ on\ the\ \‘Far\,\ Far\ Range\’\ where\ she\ tries\ her\ hand\ at\ making\ a\ living\ wrangling\ slimes\.',468,'TWD','https://store.steamp

28 , 68
28 , 21
28 , 100
28 , 22
28 , 83
28 , 46
INSERT INTO type(type_name) VALUES ('樂高');
28 , 141
28 , 27
28 , 29
28 , 115
28 , 42
28 , 6
INSERT INTO type(type_name) VALUES ('飛行');
28 , 142
28 , 32
28 , 95
28 , 35
28 , 36
28 , 1
28 , 47
INSERT INTO company(company_name) VALUES ('Flashbulb');
Trailmakers, TWD468, 極度好評
https://store.steampowered.com/app/585420/Trailmakers/?snr=1_7_7_230_150_5
['Flashbulb']
['搶先體驗', '建造', '競速', '模擬', '駕駛', '生存', '樂高', '毀滅', '沙盒', '物理', '開放世界', '冒險', '飛行', '闔家', '搞笑', '獨立', '多人', '單人', '動作']
2018-1-30

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('Prison\ Architect','\建\造\和\管\理\一\所\高\度\設\防\的\監\獄\。',438,'TWD','https://store.steampowered.com/app/233450/Prison_Architect/?snr=1_7_7_230_150_5','2015-10-6','極度好評');

29 , 96
29 , 21
29 , 22
29 , 68
INSERT INTO type(type_name) VALUES ('垂直捲軸');
29 , 143
29 , 25
29 , 26
29 , 79
29 , 80
29 , 3
29 , 29
29 , 28
29 , 31
29 , 66
29 , 9

36 , 39
36 , 64
36 , 91
36 , 153
36 , 25
36 , 14
36 , 6
36 , 74
36 , 42
INSERT INTO type(type_name) VALUES ('狙擊手');
36 , 154
36 , 43
36 , 45
36 , 62
36 , 138
36 , 36
36 , 1
36 , 47
36 , 38
INSERT INTO company(company_name) VALUES ('Rebellion');
Sniper Elite 4, TWD1248, 極度好評
https://store.steampowered.com/app/312660/Sniper_Elite_4/?snr=1_7_7_230_150_5
['Rebellion']
['戰爭', '血腥', '第三人稱射擊', '暴力', '合作', '二戰', '策略', '氛圍', '冒險', '裸露', '開放世界', '狙擊手', '第一人稱射擊', '射擊', '潛行', '戰術', '多人', '單人', '動作', '第三人稱視角']
2017-2-13

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('Ultimate\ Chicken\ Horse','Ultimate\ Chicken\ Horse\ \是\一\款\由\玩\家\自\行\創\作\關\卡\的\互\動\遊\戲\，\在\想\方\設\法\妨\礙\對\手\到\達\終\點\的\同\時\，\也\不\要\栽\在\自\己\設\下\的\陷\阱\和\障\礙\面\前\。',268,'TWD','https://store.steampowered.com/app/386940/Ultimate_Chicken_Horse/?snr=1_7_7_230_150_5','2016-3-4','壓倒性好評');

37 , 1
37 , 90
37 , 70
INSERT INTO type(type_name) VALUES ('四人單機');
37 , 15

INSERT INTO company(company_name) VALUES ('FFF Bayern');
INSERT INTO company(company_name) VALUES ('Klonk Games');
Shift Happens, TWD398, 極度好評
https://store.steampowered.com/app/359840/Shift_Happens/?snr=1_7_7_230_150_5
[' Deck13', ' FFF Bayern', 'Klonk Games']
['單機合作', '多人', '假3D', '搶先體驗', '平台', '合作', '好評原聲帶', '解謎', '獨立', '單機多人', '線上合作', '遙控器', '平台解謎', '動作']
2017-2-22

INSERT INTO game(game_name,game_introduction,game_price,game_price_currency,game_store_URL,game_release_date,game_score) VALUES ('Warhammer\:\ Vermintide\ 2','\本\作\是\廣\受\好\評\的\遊\戲\《Vermintide\》\的\續\作\。\這\款\遊\戲\將\以\震\撼\的\視\覺\效\果\和\開\創\性\的\近\戰\動\作\遊\戲\方\式\，\將\第\一\人\稱\視\角\合\作\遊\戲\推\向\新\的\高\度\。',438,'TWD','https://store.steampowered.com/app/552500/Warhammer_Vermintide_2/?snr=1_7_7_230_150_5','2018-3-8','極度好評');

45 , 118
45 , 1
45 , 64
45 , 49
45 , 91
45 , 35
45 , 46
45 , 6
45 , 120
45 , 42
INSERT INTO type(type_name) VALUES ('玩家合作');
45 , 166
45 , 43
INSERT INTO type(type_name) VALUES ('遊戲工坊');
45 , 167
45 , 14
45 , 36
45 

In [53]:
connect.close()